In [31]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [45]:
corpus = [
    "Almost everyday I wake up hating myself, why would I hate myself though? I graduated High School, have a loving and healthy family, have hobbies and online friends. Yet I find myself so paranoid and hateful of myself, why do I deserve this life I have? Sometimes I just wanna scream, feeling like I'm trapped in another person's skin, I feel like all my accomplishments are fake and I'm a useless slab of meat who'll eventually die. I don't have real life friends, post secondary, a job, a car or a license and the closest family I have is 6 hours away and barely talks to me. I want to be happy for the good things in my life but find hating myself all too easy. When I hate myself I feel selfish, like I'm so self-absorbed that I only care about myself. I just want to be happy like people online or in my family, to just always see the glass half full and not always half empty"
    "This might sound ridiculous, but I want to die, and I'm not severely depressed. I'm approaching this purely philosophically. The way I see it, my life is more suffering than joy, and that's probably not going to change. In fact, I would say this is true of most people. We spend most of our time doing things we don't want to do: working, cleaning, driving in traffic, feeling hungry, experiencing physical discomfort, etc, etc. Why would it not be better to not exist at all (I don't believe in an afterlife) rather than experience the totality of consciousness? Now other people will mourn my death, this much is true. But they're going to mourn my death regardless of when it happens. No mourning is avoided by making it happen now. If anything, I'm at a point in my life when no one is dependent on me and my death will have the smallest impact it probably ever will. So why not die?"
    "I was just prescribed it for my severe depression and anxiety."
    "Has some of you ever sometimes or all the times tell yourself that I hate myself like it just comes out out of nowhere? Or I want to die. Like I can be in deep thought and it just comes out without me thinking like it's a habit or something. I sometimes have to look around to make sure no one heard me. Does anyone ever experience this? Am I explaining this clearly enough? Edit: thanks for all the comments today wasn't the best day for me or week for that matter. Y'all comments made me feel a little better. Especially Mr. Wicked91 thanks, man!!!!!"
    "If I could just, stop existing. I would do it in a heartbeat. But I can't... I have to commit suicide in order to do that. And even then I leave footsteps. My family would be hurt. I just wish that i was never born in the first place. Then a poor family wouldn't have to worry about me. This is one of the reasons i haven't done it yet. My girlfriend would also be very upset. I just want to go without anyone being sad about me, like I never even happened. That would be so nice"
    "Bad things happen to me, and worst of all, they're inevitable. I think I'm better off dead than alive."
    "To kill myself if im ugly?"
    "Indubitably, I am not worth anything as a being, contrary to the belief of all redditors who will reply to this, I have never once done anything correct in my life, all I have ever succeeded in has been to be hurtful and disrespectful to others and taking others for granted. All I breed is catastrophe. I am not at all talented or skilled at anything in particular, I am below average in every concept known to man. I have no motivation nor intention to progress in my life. I seem to be an inconvenience, an unpleasant burden towards society. I'd say my existence is more so comparable to a tumor, I am not productive and only produce negative effects on others that unfortunately have the unbearable pain that comes from interaction with someone like me. Entrapped in this body, I dream of going into eternal sleep, a great rest that lasts to the end of time. I do not have any interest in anything anymore, daily politics bore me a great deal and I'm not at all intrigued by life at all. I don't see a reason to not follow through with it when I have no intrinsic value and no drive nor intent to live at all."
]

In [46]:
# Step 1: Tokenize the documents
# Tokenize the text into words
tokenized_documents = [word_tokenize(doc.lower()) for doc in corpus]


In [47]:
# Step 2: Remove stopwords
# Remove stop words from the tokenized documents
stop_words = set(stopwords.words('english'))
filtered_documents = [[word for word in doc if word not in stop_words] for doc in tokenized_documents]

In [48]:
# Step 3: Lemmatize words
# Lemmatize the words in the filtered documents
lemmatizer = WordNetLemmatizer()
lemmatized_documents = [[lemmatizer.lemmatize(word) for word in doc] for doc in filtered_documents]

In [49]:
# Print the preprocessed documents
print(lemmatized_documents)

[['almost', 'everyday', 'wake', 'hating', ',', 'would', 'hate', 'though', '?', 'graduated', 'high', 'school', ',', 'loving', 'healthy', 'family', ',', 'hobby', 'online', 'friend', '.', 'yet', 'find', 'paranoid', 'hateful', ',', 'deserve', 'life', '?', 'sometimes', 'wan', 'na', 'scream', ',', 'feeling', 'like', "'m", 'trapped', 'another', 'person', "'s", 'skin', ',', 'feel', 'like', 'accomplishment', 'fake', "'m", 'useless', 'slab', 'meat', "'ll", 'eventually', 'die', '.', "n't", 'real', 'life', 'friend', ',', 'post', 'secondary', ',', 'job', ',', 'car', 'license', 'closest', 'family', '6', 'hour', 'away', 'barely', 'talk', '.', 'want', 'happy', 'good', 'thing', 'life', 'find', 'hating', 'easy', '.', 'hate', 'feel', 'selfish', ',', 'like', "'m", 'self-absorbed', 'care', '.', 'want', 'happy', 'like', 'people', 'online', 'family', ',', 'always', 'see', 'glass', 'half', 'full', 'always', 'half', 'emptythis', 'might', 'sound', 'ridiculous', ',', 'want', 'die', ',', "'m", 'severely', 'depres

In [50]:
# Step 1: Create a document-term matrix
# Convert the preprocessed text data into a matrix
dictionary = corpora.Dictionary(lemmatized_documents)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in lemmatized_documents]

In [51]:
# Step 2: Choose the number of topics
# Set the number of topics you want to extract
num_topics = 4

In [52]:
# Step 3: Train the LDA model
# Train an LDA model on the document-term matrix
lda_model = LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42
)

In [53]:
# Step 4: Interpret the topics
# Print the top words in each topic
for i, topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(f"Topic {i}: {topic}")

Topic 0: 0.024*"," + 0.017*"." + 0.009*"like" + 0.008*"n't" + 0.007*"?" + 0.007*"'m" + 0.007*"would" + 0.006*"life" + 0.006*"family" + 0.006*"!"
Topic 1: 0.073*"," + 0.066*"." + 0.014*"like" + 0.014*"n't" + 0.014*"'m" + 0.013*"life" + 0.013*"would" + 0.013*"want" + 0.013*"?" + 0.010*"anything"
Topic 2: 0.048*"." + 0.043*"," + 0.014*"like" + 0.013*"'m" + 0.012*"?" + 0.012*"n't" + 0.011*"would" + 0.011*"life" + 0.008*"family" + 0.007*"want"
Topic 3: 0.058*"." + 0.048*"," + 0.014*"would" + 0.014*"like" + 0.014*"life" + 0.013*"?" + 0.012*"'m" + 0.011*"n't" + 0.009*"!" + 0.008*"die"


In [54]:
# Compute the coherence score
coherence_model_lda = CoherenceModel(
    model=lda_model,
    texts=lemmatized_documents,
    dictionary=dictionary,
    coherence='c_v'
)

coherence_lda = coherence_model_lda.get_coherence()

print(f"Coherence Score: {coherence_lda}")

Coherence Score: 0.20935659920622937
